# 360dga数据预处理

In [1]:
import pandas as pd
import re
def read_txt(filePath):
    with open(filePath,"r") as f:
        data = f.readlines()
    return [d.strip() for d in data]

In [2]:
dga_360 = read_txt("origin_data/360dga.txt")
dga_360 = [d for d in dga_360 if re.match("#.*",d)==None and len(d) > 1]
dga_360 = [(d.split('\t')[:2]) for d in dga_360]
dga_360_df = pd.DataFrame(dga_360,columns=["family","urls"])

In [3]:
dga_360_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1244514 entries, 0 to 1244513
Data columns (total 2 columns):
 #   Column  Non-Null Count    Dtype 
---  ------  --------------    ----- 
 0   family  1244514 non-null  object
 1   urls    1244514 non-null  object
dtypes: object(2)
memory usage: 19.0+ MB


In [4]:
import pickle
with open("data/dga_360","wb") as f:
    pickle.dump(dga_360_df,f)

# alexa-top-100w数据预处理

In [5]:
alexa_2019 = pd.read_excel("origin_data/2019-alexa-top-100w.xlsx",header=None)
alexa_2019.columns = ["id","urls"]
alexa_2019 = alexa_2019.drop("id",axis=1)

In [6]:
alexa_2019.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000000 entries, 0 to 999999
Data columns (total 1 columns):
 #   Column  Non-Null Count    Dtype 
---  ------  --------------    ----- 
 0   urls    1000000 non-null  object
dtypes: object(1)
memory usage: 7.6+ MB


In [7]:
with open("data/alexa_2019","wb") as f:
    pickle.dump(alexa_2019,f)

# 整合为数据集

## 合并360dga与alexa

In [8]:
import pickle
with open("data/dga_360","rb") as f:
    negative = pickle.load(f)

with open("data/alexa_2019","rb") as f:
    positive = pickle.load(f)
    
from sklearn.utils import shuffle
import pandas as pd
negative = shuffle(negative)
positive = shuffle(positive)
negative = negative.drop("family",axis = 1)
negative['label'] = 1
positive['label'] = 0
data = pd.concat([positive,negative])
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2244514 entries, 399606 to 638150
Data columns (total 2 columns):
 #   Column  Dtype 
---  ------  ----- 
 0   urls    object
 1   label   int64 
dtypes: int64(1), object(1)
memory usage: 51.4+ MB


## 切分数据集为训练、验证和测试

In [9]:
X = data["urls"]
y = data["label"]
from sklearn.model_selection import train_test_split
data = train_test_split(X, y, test_size=0.1, random_state=2020,stratify=y)
test = data[1],data[3]

In [10]:
# 继续切分训练集和验证集
data = train_test_split(data[0], data[2], test_size=0.1, random_state=2020,stratify=data[2])
train = data[0],data[2]
val = data[1],data[3]

In [11]:
with open("data/train_data","wb") as f:
    pickle.dump(train,f)
    
with open("data/text_data","wb") as f:
    pickle.dump(test,f)
    
with open("data/val_data","wb") as f:
    pickle.dump(val,f)